In [54]:
from scipy.io.wavfile import read, write
import torchaudio
import torch
from librosa.util import normalize
from librosa.filters import mel as librosa_mel_fn
import numpy as np
import librosa
import librosa.display
from tqdm import tqdm
import os
import soundfile as sf
import matplotlib.pyplot as plt
from transformers import CLIPTextModel, CLIPTokenizer, T5Tokenizer, T5EncoderModel, AutoTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, PNDMScheduler
%matplotlib inline

In [ ]:
temp_vae_path = "/blob/v-yuancwang/AudioEditing/VAE_GAN/checkpoint-50000"
vae = AutoencoderKL.from_pretrained(temp_vae_path, subfolder="vae")
vae.requires_grad_(False)

In [ ]:
torch_device = "cuda:2"
vae.to(torch_device)

In [ ]:
test_data_path = "/blob/v-yuancwang/audio_editing_data/replacement/mel"
test_mel = np.load(os.path.join(test_data_path, "replacementgen14_1.npy"))
plt.imshow(test_mel)

In [ ]:
librosa.display.specshow(test_mel, sr=16000, hop_length=256, x_axis='time', y_axis='mel')
plt.show()

In [ ]:
with torch.no_grad():
    res_mel = torch.Tensor([[test_mel]]).to(torch.float32)
    res_mel = res_mel.to(torch_device)
    print(res_mel.shape)
    res_mel = vae(res_mel, sample_posterior=True).sample
    res_mel = res_mel.cpu().numpy()[0,0,:,:]
plt.imshow(res_mel)

In [ ]:
librosa.display.specshow(res_mel, sr=16000, hop_length=256, x_axis='time', y_axis='mel')
plt.show()

In [59]:
import shutil

In [60]:
wav_path = "/blob/v-yuancwang/audio_editing_data/audiocaps/wav"
mel_path = "/blob/v-yuancwang/audio_editing_data/audiocaps/mel"
with open("/home/v-yuancwang/AudioEditing/metadatas/audiocaps_train_metadata.jsonl", "r") as f:
    lines = f.readlines()
lines = [eval(line) for line in lines]
for i in tqdm(range(len(lines))[:200]):
    wav_name = lines[i]["file_name"]
    # shutil.copyfile(os.path.join(wav_path, wav_name), os.path.join("/home/v-yuancwang/AudioEditing/hifigan_test_vae_gt", wav_name))
    test_mel = np.load(os.path.join(mel_path, wav_name.replace(".wav", ".npy")))
    with torch.no_grad():
        res_mel = torch.Tensor([[test_mel]]).to(torch.float32)
        res_mel = res_mel.to(torch_device)
        res_mel = vae(res_mel, sample_posterior=True).sample
        res_mel = res_mel.cpu().numpy()[0,0,:,:]
    np.save(os.path.join("/home/v-yuancwang/AudioEditing/hifigan_test_vae",wav_name.replace(".wav", ".npy")), res_mel)

100%|██████████| 200/200 [00:22<00:00,  8.82it/s]
